In [1]:
import pandas as pd
import numpy as np
import logging
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)





In [3]:

# Load the datasets
logger.info("Loading datasets...")

# Load test data
test_df = pd.read_csv('test.csv')
logger.info(f"Test data loaded: {test_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Test data columns: {list(test_df.columns)}")

# Load store data
store_df = pd.read_csv('store.csv')
logger.info(f"Store data loaded: {store_df.shape}")
print(f"Store data shape: {store_df.shape}")
print(f"Store data columns: {list(store_df.columns)}")

2025-06-26 05:10:18,018 - INFO - Loading datasets...
2025-06-26 05:10:18,119 - INFO - Test data loaded: (41088, 8)
2025-06-26 05:10:18,134 - INFO - Store data loaded: (1115, 10)


Test data shape: (41088, 8)
Test data columns: ['Id', 'Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']
Store data shape: (1115, 10)
Store data columns: ['Store', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']


In [5]:
# Display first few rows
print("\n=== Test Data Sample ===")
print(test_df.head())

print("\n=== Store Data Sample ===")
print(store_df.head())


=== Test Data Sample ===
   Id  Store  DayOfWeek        Date  Open  Promo StateHoliday  SchoolHoliday
0   1      1          4  2015-09-17   1.0      1            0              0
1   2      3          4  2015-09-17   1.0      1            0              0
2   3      7          4  2015-09-17   1.0      1            0              0
3   4      8          4  2015-09-17   1.0      1            0              0
4   5      9          4  2015-09-17   1.0      1            0              0

=== Store Data Sample ===
   Store StoreType Assortment  CompetitionDistance  CompetitionOpenSinceMonth  \
0      1         c          a               1270.0                        9.0   
1      2         a          a                570.0                       11.0   
2      3         a          a              14130.0                       12.0   
3      4         c          c                620.0                        9.0   
4      5         a          a              29910.0                        4.0   

In [7]:
print("\n=== DATA QUALITY ANALYSIS ===")

# Check missing values in test data
logger.info("Checking missing values in test data...")
test_missing = test_df.isnull().sum()
test_missing_percent = (test_missing / len(test_df)) * 100

print("\nMissing values in Test data:")
for col in test_df.columns:
    if test_missing[col] > 0:
        print(f"{col}: {test_missing[col]} ({test_missing_percent[col]:.2f}%)")


2025-06-26 05:10:57,668 - INFO - Checking missing values in test data...



=== DATA QUALITY ANALYSIS ===

Missing values in Test data:
Open: 11 (0.03%)


In [9]:
# Check missing values in store data
logger.info("Checking missing values in store data...")
store_missing = store_df.isnull().sum()
store_missing_percent = (store_missing / len(store_df)) * 100

print("\nMissing values in Store data:")
for col in store_df.columns:
    if store_missing[col] > 0:
        print(f"{col}: {store_missing[col]} ({store_missing_percent[col]:.2f}%)")


2025-06-26 05:11:17,204 - INFO - Checking missing values in store data...



Missing values in Store data:
CompetitionDistance: 3 (0.27%)
CompetitionOpenSinceMonth: 354 (31.75%)
CompetitionOpenSinceYear: 354 (31.75%)
Promo2SinceWeek: 544 (48.79%)
Promo2SinceYear: 544 (48.79%)
PromoInterval: 544 (48.79%)


In [11]:
# Check data types
print("\n=== DATA TYPES ===")
print("\nTest data types:")
print(test_df.dtypes)

print("\nStore data types:")
print(store_df.dtypes)


=== DATA TYPES ===

Test data types:
Id                 int64
Store              int64
DayOfWeek          int64
Date              object
Open             float64
Promo              int64
StateHoliday      object
SchoolHoliday      int64
dtype: object

Store data types:
Store                          int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object


In [13]:
# ===========================================
# DATA CLEANING - TEST DATA
# ===========================================

logger.info("Starting data cleaning for test data...")

# Make copies to preserve original data
test_cleaned = test_df.copy()
store_cleaned = store_df.copy()

# Convert Date column to datetime
test_cleaned['Date'] = pd.to_datetime(test_cleaned['Date'])
logger.info("Converted Date column to datetime")


2025-06-26 05:11:56,026 - INFO - Starting data cleaning for test data...
2025-06-26 05:11:56,073 - INFO - Converted Date column to datetime


In [15]:
# Handle missing values in test data

if 'Open' in test_cleaned.columns:
    test_cleaned['Open'] = test_cleaned['Open'].fillna(1)
    logger.info("Filled missing Open values with 1")


2025-06-26 05:12:21,217 - INFO - Filled missing Open values with 1


In [17]:
# ===========================================
# DATA CLEANING - STORE DATA
# ===========================================

logger.info("Starting data cleaning for store data...")

# Handle missing values in store data
# CompetitionDistance - fill with median or a large value
if store_cleaned['CompetitionDistance'].isnull().sum() > 0:
    median_distance = store_cleaned['CompetitionDistance'].median()
    store_cleaned['CompetitionDistance'] = store_cleaned['CompetitionDistance'].fillna(median_distance)
    logger.info(f"Filled missing CompetitionDistance with median: {median_distance}")

2025-06-26 05:12:43,149 - INFO - Starting data cleaning for store data...
2025-06-26 05:12:43,161 - INFO - Filled missing CompetitionDistance with median: 2325.0


In [21]:
# CompetitionOpenSinceMonth and CompetitionOpenSinceYear
# Fill with 0 or a default value indicating no competition data
competition_cols = ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']
for col in competition_cols:
    if col in store_cleaned.columns:
        store_cleaned[col] = store_cleaned[col].fillna(0)
        logger.info(f"Filled missing {col} with 0")

# Promo2SinceWeek and Promo2SinceYear
# Fill with 0 indicating no Promo2 participation
promo2_cols = ['Promo2SinceWeek', 'Promo2SinceYear']
for col in promo2_cols:
    if col in store_cleaned.columns:
        store_cleaned[col] = store_cleaned[col].fillna(0)
        logger.info(f"Filled missing {col} with 0")

# PromoInterval - fill with empty string
if 'PromoInterval' in store_cleaned.columns:
    store_cleaned['PromoInterval'] = store_cleaned['PromoInterval'].fillna('')
    logger.info("Filled missing PromoInterval with empty string")

# ===========================================
# MERGE TEST AND STORE DATA
# ===========================================

logger.info("Merging test and store data...")
# Merge test data with store data on Store column
merged_df = test_cleaned.merge(store_cleaned, on='Store', how='left')
logger.info(f"Merged data shape: {merged_df.shape}")

print(f"\nMerged data shape: {merged_df.shape}")
print(f"Merged data columns: {list(merged_df.columns)}")


2025-06-26 05:13:40,486 - INFO - Filled missing CompetitionOpenSinceMonth with 0
2025-06-26 05:13:40,494 - INFO - Filled missing CompetitionOpenSinceYear with 0
2025-06-26 05:13:40,494 - INFO - Filled missing Promo2SinceWeek with 0
2025-06-26 05:13:40,499 - INFO - Filled missing Promo2SinceYear with 0
2025-06-26 05:13:40,504 - INFO - Filled missing PromoInterval with empty string
2025-06-26 05:13:40,504 - INFO - Merging test and store data...
2025-06-26 05:13:40,552 - INFO - Merged data shape: (41088, 17)



Merged data shape: (41088, 17)
Merged data columns: ['Id', 'Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']


In [23]:
# Check for any missing values after merge
merged_missing = merged_df.isnull().sum()
print("\nMissing values after merge:")
for col in merged_df.columns:
    if merged_missing[col] > 0:
        print(f"{col}: {merged_missing[col]} ({(merged_missing[col]/len(merged_df)*100):.2f}%)")



Missing values after merge:


In [25]:
# ===========================================
# FEATURE ENGINEERING FROM DATE
# ===========================================

logger.info("Creating date-based features...")

# Extract date components
merged_df['Year'] = merged_df['Date'].dt.year
merged_df['Month'] = merged_df['Date'].dt.month
merged_df['Day'] = merged_df['Date'].dt.day
merged_df['DayOfWeek'] = merged_df['Date'].dt.dayofweek  # 0=Monday, 6=Sunday
merged_df['WeekOfYear'] = merged_df['Date'].dt.isocalendar().week
merged_df['Quarter'] = merged_df['Date'].dt.quarter

# Create weekend indicator
merged_df['IsWeekend'] = (merged_df['DayOfWeek'] >= 5).astype(int)

# Create month indicators
merged_df['IsMonthStart'] = merged_df['Date'].dt.is_month_start.astype(int)
merged_df['IsMonthEnd'] = merged_df['Date'].dt.is_month_end.astype(int)

# Day of month categories
merged_df['DayOfMonth'] = merged_df['Date'].dt.day
merged_df['IsBeginningOfMonth'] = (merged_df['DayOfMonth'] <= 10).astype(int)
merged_df['IsMiddleOfMonth'] = ((merged_df['DayOfMonth'] > 10) & (merged_df['DayOfMonth'] <= 20)).astype(int)
merged_df['IsEndOfMonth'] = (merged_df['DayOfMonth'] > 20).astype(int)

logger.info("Created date-based features: Year, Month, Day, DayOfWeek, WeekOfYear, Quarter, IsWeekend, etc.")



2025-06-26 05:14:47,943 - INFO - Creating date-based features...
2025-06-26 05:14:48,018 - INFO - Created date-based features: Year, Month, Day, DayOfWeek, WeekOfYear, Quarter, IsWeekend, etc.


In [27]:
# ===========================================
# LABEL ENCODING FOR CATEGORICAL VARIABLES
# ===========================================

logger.info("Applying label encoding to categorical variables...")

# Initialize label encoders dictionary
label_encoders = {}

# Categorical columns that need label encoding
categorical_columns = ['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval']

# Apply label encoding
for col in categorical_columns:
    if col in merged_df.columns:
        le = LabelEncoder()
        # Handle missing values by filling with 'Unknown' before encoding
        merged_df[col] = merged_df[col].fillna('Unknown')
        merged_df[f'{col}_Encoded'] = le.fit_transform(merged_df[col].astype(str))
        label_encoders[col] = le
        
        # Display encoding mapping
        print(f"\nLabel encoding for {col}:")
        for i, label in enumerate(le.classes_):
            print(f"  {label} -> {i}")
        
        logger.info(f"Applied label encoding to {col}")

2025-06-26 05:15:13,979 - INFO - Applying label encoding to categorical variables...
2025-06-26 05:15:14,006 - INFO - Applied label encoding to StateHoliday
2025-06-26 05:15:14,033 - INFO - Applied label encoding to StoreType
2025-06-26 05:15:14,110 - INFO - Applied label encoding to Assortment
2025-06-26 05:15:14,139 - INFO - Applied label encoding to PromoInterval



Label encoding for StateHoliday:
  0 -> 0
  a -> 1

Label encoding for StoreType:
  a -> 0
  b -> 1
  c -> 2
  d -> 3

Label encoding for Assortment:
  a -> 0
  b -> 1
  c -> 2

Label encoding for PromoInterval:
   -> 0
  Feb,May,Aug,Nov -> 1
  Jan,Apr,Jul,Oct -> 2
  Mar,Jun,Sept,Dec -> 3


In [29]:
# Competition features
# Years since competition opened
current_year = merged_df['Year'].max()
merged_df['CompetitionOpenSinceYear'] = merged_df['CompetitionOpenSinceYear'].replace(0, current_year)
merged_df['YearsSinceCompetitionOpen'] = merged_df['Year'] - merged_df['CompetitionOpenSinceYear']
merged_df['YearsSinceCompetitionOpen'] = merged_df['YearsSinceCompetitionOpen'].clip(lower=0)

# Months since competition opened
merged_df['MonthsSinceCompetitionOpen'] = (merged_df['Year'] - merged_df['CompetitionOpenSinceYear']) * 12 + \
                                         (merged_df['Month'] - merged_df['CompetitionOpenSinceMonth'])
merged_df['MonthsSinceCompetitionOpen'] = merged_df['MonthsSinceCompetitionOpen'].clip(lower=0)

# Promo2 features
# Weeks since Promo2 started
merged_df['Promo2SinceYear'] = merged_df['Promo2SinceYear'].replace(0, current_year)
merged_df['WeeksSincePromo2Start'] = (merged_df['Year'] - merged_df['Promo2SinceYear']) * 52 + \
                                    (merged_df['WeekOfYear'] - merged_df['Promo2SinceWeek'])
merged_df['WeeksSincePromo2Start'] = merged_df['WeeksSincePromo2Start'].clip(lower=0)

# Promo2 active indicator
merged_df['IsPromo2Active'] = merged_df['Promo2'].copy()

# Check if current month is in PromoInterval
def is_promo_month(row):
    if pd.isna(row['PromoInterval']) or row['PromoInterval'] == '':
        return 0
    
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    current_month = month_names[row['Month'] - 1]
    
    return 1 if current_month in row['PromoInterval'] else 0

merged_df['IsPromoMonth'] = merged_df.apply(is_promo_month, axis=1)

logger.info("Created competition and promo-related features")


2025-06-26 05:15:44,773 - INFO - Created competition and promo-related features


In [33]:
logger.info("Final data cleaning and validation...")

# Handle any remaining missing values
numeric_columns = merged_df.select_dtypes(include=[np.number]).columns
for col in numeric_columns:
    if merged_df[col].isnull().sum() > 0:
        merged_df[col] = merged_df[col].fillna(merged_df[col].median())
        logger.info(f"Filled remaining missing values in {col} with median")

# Convert boolean columns to int
bool_columns = merged_df.select_dtypes(include=['bool']).columns
for col in bool_columns:
    merged_df[col] = merged_df[col].astype(int)



print("\n=== FINAL CLEANED DATA SUMMARY ===")
print(f"Final data shape: {merged_df.shape}")
print(f"Total columns: {len(merged_df.columns)}")

# Check for missing values in final dataset
final_missing = merged_df.isnull().sum()
print(f"\nRemaining missing values: {final_missing.sum()}")

if final_missing.sum() > 0:
    print("Columns with missing values:")
    for col in merged_df.columns:
        if final_missing[col] > 0:
            print(f"  {col}: {final_missing[col]}")

2025-06-26 05:16:19,068 - INFO - Final data cleaning and validation...



=== FINAL CLEANED DATA SUMMARY ===
Final data shape: (41088, 38)
Total columns: 38

Remaining missing values: 0


In [35]:
# Display data types
print("\nFinal data types:")
print(merged_df.dtypes.value_counts())

# Display sample of cleaned data
print("\n=== SAMPLE OF CLEANED DATA ===")
print(merged_df.head())

# Save cleaned data
merged_df.to_csv('cleaned_test_data.csv', index=False)
logger.info("Saved cleaned data to 'cleaned_test_data.csv'")


Final data types:
int32             16
float64            8
int64              7
object             4
datetime64[ns]     1
UInt32             1
Float64            1
Name: count, dtype: int64

=== SAMPLE OF CLEANED DATA ===
   Id  Store  DayOfWeek       Date  Open  Promo StateHoliday  SchoolHoliday  \
0   1      1          3 2015-09-17   1.0      1            0              0   
1   2      3          3 2015-09-17   1.0      1            0              0   
2   3      7          3 2015-09-17   1.0      1            0              0   
3   4      8          3 2015-09-17   1.0      1            0              0   
4   5      9          3 2015-09-17   1.0      1            0              0   

  StoreType Assortment  ...  IsEndOfMonth  StateHoliday_Encoded  \
0         c          a  ...             0                     0   
1         a          a  ...             0                     0   
2         a          c  ...             0                     0   
3         a          a  ...      

2025-06-26 05:16:36,874 - INFO - Saved cleaned data to 'cleaned_test_data.csv'
